# Model recommendation with lighfm

### Import libraries

In [1]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

### Defining variables

In [2]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [ ]:

TEST_PERCENTAGE = 0.20
LEARNING_RATE = 0.1
NUM_EPOCHS = 80
NUM_COMPONENTS = 10
NUM_THREADS = 3
ALPHA_REG_L2 = 1e-3
MAX_SAMPLED = 10
SEED = 42

### Retrieve data

In [4]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [5]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=2000000)
df_ratings

,Unnamed: 0,userId,history,userType,score
0,0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477
1,1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501
2,2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861
3,3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271
4,4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852
...,...,...,...,...,...
1999995,1999995,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,64323b0f-d998-4f8d-a9ff-0e8c4f8859a0,Logged,0.589154
1999996,1999996,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,563fae3d-f36e-48a3-b56d-bf309aba3d2f,Logged,0.331947
1999997,1999997,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,eaa13689-5f22-45a4-b87f-e54193d1f3a9,Logged,0.190049
1999998,1999998,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,005a743a-0a81-4dd4-ba1f-6dc63cc33a34,Logged,0.262472


In [6]:
df_ratings.drop(columns=["Unnamed: 0"],inplace=True)
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userId    string  
 1   history   string  
 2   userType  category
 3   score     Float32 
dtypes: Float32(1), category(1), string(2)
memory usage: 42.0 MB


In [7]:
df_ratings.describe()

,score
count,2000000.0
mean,1.614625
std,0.86737
min,0.013183
25%,0.933404
50%,1.574703
75%,2.207815
max,4.990057


In [8]:
dtype_df_items = {
"page" : 'string',
"url" : 'string',
"issued" : 'string',
"modified" : 'string',
"title" : 'string',
"body" : 'string',
"caption" : 'string',
"age_in_days" : 'UInt32',
"age_exp" : 'Float32',
"age_exp_normalized" : 'Float32',
"ageCategories" : 'category'
}

In [9]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"], dtype=dtype_df_items)
df_news.drop(columns=["Unnamed: 0"],inplace=True)

In [10]:
df_news.head(3)

,page,age_exp_normalized,ageCategories
0,7371a9b5-5824-4c57-8704-00a74feebe79,0.151439,very-old
1,7a5ea08f-4583-49e2-ba52-a71999443f7b,0.140788,very-old
2,6afc8bbb-4f36-43d5-8a44-a2917df5621a,0.12261,very-old


In [11]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255603 entries, 0 to 255602
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   page                255603 non-null  string  
 1   age_exp_normalized  255603 non-null  Float32 
 2   ageCategories       255603 non-null  category
dtypes: Float32(1), category(1), string(1)
memory usage: 3.4 MB


In [12]:
df_merged = pd.merge(df_ratings, df_news, left_on='history', right_on='page', how='left')
df_merged.drop(columns=["page"],inplace=True) # 'page' is the same as 'history'

In [13]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 6 columns):
 #   Column              Dtype   
---  ------              -----   
 0   userId              string  
 1   history             string  
 2   userType            category
 3   score               Float32 
 4   age_exp_normalized  Float32 
 5   ageCategories       category
dtypes: Float32(2), category(2), string(2)
memory usage: 53.4 MB


In [14]:
df_merged.head()

,userId,history,userType,score,age_exp_normalized,ageCategories
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477,0.980416,recent
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501,0.613061,mid
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861,0.880859,recent
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271,0.945895,recent
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852,0.13293,very-old


In [15]:
df_merged["check_hifens"] = df_merged["history"].str.split('-').apply(lambda x : len(x) == 5).astype("bool")
df_merged["check_size"] = df_merged["history"].str.replace(r'-', '', regex=True).apply(lambda x : len(x) == 32).astype("bool")
df_merged["check_chars"] = df_merged["history"].str.replace(r'-', '', regex=True).str.replace(r'[a-f0-9]', '', regex=True).apply(lambda x : x == '').astype("bool")
df_merged["check_history"] = (df_merged["check_hifens"] & df_merged["check_size"] & df_merged["check_chars"]).astype("bool")
df_merged["check_history"].value_counts()

check_history
True     1995128
False       4872
Name: count, dtype: int64

In [16]:
df_merged = df_merged[~df_merged["check_history"]==False]
df_merged

,userId,history,userType,score,age_exp_normalized,ageCategories,check_hifens,check_size,check_chars,check_history
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,2.216477,0.980416,recent,True,True,True,True
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,2.429501,0.613061,mid,True,True,True,True
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,1.794861,0.880859,recent,True,True,True,True
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,1.68271,0.945895,recent,True,True,True,True
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,2.266852,0.13293,very-old,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
1999995,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,64323b0f-d998-4f8d-a9ff-0e8c4f8859a0,Logged,0.589154,0.95782,recent,True,True,True,True
1999996,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,563fae3d-f36e-48a3-b56d-bf309aba3d2f,Logged,0.331947,0.954396,recent,True,True,True,True
1999997,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,eaa13689-5f22-45a4-b87f-e54193d1f3a9,Logged,0.190049,0.959538,recent,True,True,True,True
1999998,dbce7688568e5ded880796d9f0a1327fa4ae6b2401d242...,005a743a-0a81-4dd4-ba1f-6dc63cc33a34,Logged,0.262472,0.660257,mid,True,True,True,True


In [ ]:
w_initial_score = 1
w_age_norm = 1-w_initial_score
df_merged["score_init"] = df_merged["score"]*w_initial_score + df_merged["age_exp_normalized"]*w_age_norm
df_merged["score_fn"] = df_merged["score_init"].apply(lambda x: np.power(x,0.33))
df_merged["score_norm"] = (df_merged["score_fn"]-df_merged["score_fn"].min())/(df_merged["score_fn"].max()-df_merged["score_fn"].min())
df_merged["score_1_to_5"] = df_merged["score_norm"]*4+1
df_merged["score_1_to_5_int"] = df_merged["score_1_to_5"].round(0).astype("UInt16")
df_merged.describe()

,score,age_exp_normalized,score_init,score_fn,score_norm,score_1_to_5,score_1_to_5_int
count,1995128.0,1995128.0,1995128.0,1.995128e+06,1.995128e+06,1.995128e+06,1995128.0
mean,1.614683,0.90185,1.507758,1.109664e+00,5.781333e-01,3.312533e+00,3.304141
std,0.867461,0.181615,0.740301,2.026401e-01,1.645527e-01,6.582107e-01,0.728989
min,0.013183,0.106769,0.061176,3.977154e-01,0.000000e+00,1.000000e+00,1.0
25%,0.933274,0.934135,0.925959,9.749341e-01,4.687269e-01,2.874908e+00,3.0
50%,1.574852,0.956106,1.47424,1.136652e+00,6.000491e-01,3.400196e+00,3.0
75%,2.207985,0.978657,2.014539,1.260022e+00,7.002306e-01,3.800922e+00,4.0
max,4.990057,1.0,4.388611,1.629176e+00,1.000000e+00,5.000000e+00,5.0


In [ ]:
age_exp_normalized

In [18]:
column_score_to_use = "score_1_to_5"
df_merged = df_merged[["userId","history","userType",column_score_to_use]]
df_merged.rename(columns={column_score_to_use:"score"},inplace=True)

df_merged.rename(columns={"ageCategories" : "historyFreshnessNormalized"},inplace=True)
df_merged.describe()

/tmp/ipykernel_127484/342061991.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged.rename(columns={column_score_to_use:"score"},inplace=True)
/tmp/ipykernel_127484/342061991.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged.rename(columns={"ageCategories" : "historyFreshnessNormalized"},inplace=True)


,score
count,1.995128e+06
mean,3.312533e+00
std,6.582107e-01
min,1.000000e+00
25%,2.874908e+00
50%,3.400196e+00
75%,3.800922e+00
max,5.000000e+00


In [19]:
# df_merged.head()
df_merged.head()

,userId,history,userType,score
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,80aa7bb2-adce-4a55-9711-912c407927a1,Non-Logged,3.811974
1,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,d9e5f15d-b441-4d8b-bee4-462b106d3916,Non-Logged,3.894278
2,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,e273dba4-136c-45fb-bdd6-0cc57b13aaf0,Non-Logged,3.545945
3,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,a0562805-c7d1-4ffd-b622-87c50ae006f4,Non-Logged,3.479406
4,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,233f8238-2ce0-470f-a9d5-0e0ac530382a,Non-Logged,3.754961


In [20]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1995128 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column    Dtype   
---  ------    -----   
 0   userId    string  
 1   history   string  
 2   userType  category
 3   score     float64 
dtypes: category(1), float64(1), string(2)
memory usage: 62.8 MB


### Prepare data

Before fitting the LightFM model, we need to create an instance of `Dataset` which holds the interaction matrix.

In [21]:
dataset = Dataset()

# Get unique values for users, items, and user features
unique_users = df_merged["userId"].unique()
unique_items = df_merged["history"].unique()
unique_user_features = df_merged["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset.fit(
    users=unique_users,
    items=unique_items,
    user_features=unique_user_features  # Register user features
)


In [22]:
(interactions, weights) = dataset.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_merged.iterrows()
])


In [23]:
user_features_list = [
    (row.userId, [row.userType])  
    for _, row in df_merged.iterrows()
]

user_features = dataset.build_user_features(user_features_list)


LightLM works slightly differently compared to other packages as it expects the train and test sets to have same dimension. Therefore the conventional train test split will not work.

The package has included the `cross_validation.random_train_test_split` method to split the interaction data and splits it into two disjoint training and test sets. 

However, note that **it does not validate the interactions in the test set to guarantee all items and users have historical interactions in the training set**. Therefore this may result into a partial cold-start problem in the test set.

In [24]:
# Split train and test sets (80/20 split)
train, test = cross_validation.random_train_test_split(interactions, test_percentage=TEST_PERCENTAGE, random_state=SEED)
train_weights, test_weights  = cross_validation.random_train_test_split(weights, test_percentage=TEST_PERCENTAGE, random_state=SEED)


Double check the size of both the train and test sets.

In [25]:
print(f"Shape of train interactions: {train.shape}")
print(f"Shape of test interactions: {test.shape}")

Shape of train interactions: (168028, 130285)
Shape of test interactions: (168028, 130285)


In [26]:
print(f"Shape of train interactions: {train_weights.shape}")
print(f"Shape of test interactions: {test_weights.shape}")

Shape of train interactions: (168028, 130285)
Shape of test interactions: (168028, 130285)


### Fit the LightFM model

In this notebook, the LightFM model will be using the weighted Approximate-Rank Pairwise (WARP) as the loss. Further explanation on the topic can be found [here](https://making.lyst.com/lightfm/docs/examples/warp_loss.html#learning-to-rank-using-the-warp-loss).


In general, it maximises the rank of positive examples by repeatedly sampling negative examples until a rank violation has been located. This approach is recommended when only positive interactions are present.

The LightFM model can be fitted with the following code:

In [27]:
model = LightFM(no_components=NUM_COMPONENTS,loss="warp",learning_rate=LEARNING_RATE,user_alpha=ALPHA_REG_L2,max_sampled=MAX_SAMPLED,random_state=np.random.RandomState(SEED))  # Weighted Approximate-Rank Pairwise (WARP) loss
model.fit(train, sample_weight=train_weights, epochs=NUM_EPOCHS, num_threads=NUM_THREADS, user_features=user_features)


### Evaluate model

In [28]:
# Import the evaluation routines
from lightfm.evaluation import auc_score

# Compute evaluation metrics
auc_train = auc_score(model, train, user_features=user_features, num_threads=NUM_THREADS).mean()
auc_test = auc_score(model, test, train_interactions=train, user_features=user_features, num_threads=NUM_THREADS).mean()

# Print evaluation results
print(f"AUC test Score: {auc_test:.4f}")
print(f"AUC train Score: {auc_train:.4f}")

AUC test Score: 0.7404
AUC train Score: 0.7029


In [29]:
# # Measure how well it did in the Test period
# for metric in [precision_at_k, recall_at_k]:
#     # Get the precision and recall for Train and Test
#     for data, name in [(train, "Train"), (test, "Test ")]:
#         print(f"{name} {metric.__name__}: %.2f" % 
#               metric(model,
#                          data, 
#                          k=10).mean())

### Save pkls to serve model

In [30]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

In [31]:
item_id_map_reverse = {v: k for k, v in item_id_map.items()}

In [32]:
interactions_shape = interactions.shape

In [33]:
import pickle

pickle.dump(model, open('artifacts/lightfm_model.pkl', 'wb'))
pickle.dump(user_id_map, open('artifacts/user_id_map.pkl', 'wb'))
pickle.dump(item_id_map_reverse, open('artifacts/item_id_map_reverse.pkl', 'wb'))
pickle.dump(user_feature_map, open('artifacts/user_feature_map.pkl', 'wb'))
pickle.dump(interactions_shape, open('artifacts/interactions_shape.pkl', 'wb'))

In [34]:
import pickle
import numpy as np
import pandas as pd

In [35]:
loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))
loaded_interactions_shape = pickle.load(open('artifacts/interactions_shape.pkl', 'rb'))

### Interaction and Weight Matrices

In [36]:
# print(weights[:5,:5].todense())

### Scored Matrix

In [37]:
# # Force lightFM to create predictions for all users and all items
# import numpy as np
# n_items = 5
# n_users = 7

# print([np.full((n_items, ), 1), np.full((n_items, ), 2)])
# print(np.concatenate([np.full((n_items, ), 1), np.full((n_items, ), 2)]))
# print(np.arange((n_items)))

In [38]:
# n_items = 5
# n_users = 7

# scoring_user_ids = np.concatenate([np.full((n_items, ), i) for i in range(n_users)]) # repeat user ID for number of prods 
# scoring_item_ids = np.concatenate([np.arange(n_items) for i in range(n_users)]) # repeat entire range of item IDs x number of user
# print(scoring_user_ids)
# print(scoring_item_ids)

In [39]:
loaded_n_users, loaded_n_items = loaded_interactions_shape

In [40]:
# items_idx_to_score = np.arange(loaded_n_items)
# scores_matrix_dtype = np.int16

# scores = np.zeros((loaded_n_users,loaded_n_items),dtype=scores_matrix_dtype)

# for user in range(1000):
#     scores[user,] = loaded_model.predict(user_ids = user, item_ids = items_idx_to_score)
#     if user%1000 == 0:
#         print(f"iteration = {user}")

# # scores = model.predict(user_ids = scoring_user_ids, 
# #                                      item_ids = scoring_item_ids)
# # scores = scores.reshape(-1, n_items) # get 1 row per user
# recommendations = pd.DataFrame(scores).astype("Int16")
# recommendations.shape

# # Have a look at the predicted scores for the first 5 users and first 5 items
# recommendations.iloc[:5,:5] 

### Make predictions to known and unknowm on same recommendation function with pkls

In [41]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [42]:
def sample_recommendation(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(loaded_n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(loaded_n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:5]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    print("Top 5 recommended items:")

    for x in top_5_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["page"].values[0])


In [43]:
# predict for known user
user_feature_list = ['userType:Logged']
user_hash = '5f5e17781fc2ec0ddcfb2e9356e61c5d3d4b0b3c8fabd20917feb9e807463856'
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

new user feature encountered 'userType:Logged'
Top 5 recommended items:
        d2593c3d-2347-40d9-948c-b6065e8459a9
        f6b5d170-48b9-4f8e-88d4-c84b6668f3bd
        1f32787b-de2b-49be-8c20-ddaeae34cc22
        6a83890a-d9e9-4f6b-a6c6-90d031785bbf
        f0a78e58-ec7e-494c-9462-fbd6446a9a89


In [44]:
# predict for unknown user
user_feature_list = ['userType:Non-Logged']
user_hash = ''
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

new user feature encountered 'userType:Non-Logged'
Top 5 recommended items:
        d2593c3d-2347-40d9-948c-b6065e8459a9
        f6b5d170-48b9-4f8e-88d4-c84b6668f3bd
        1f32787b-de2b-49be-8c20-ddaeae34cc22
        6a83890a-d9e9-4f6b-a6c6-90d031785bbf
        f0a78e58-ec7e-494c-9462-fbd6446a9a89


In [45]:
df_merged.loc[0,'userId']

'fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27288434101e7252ef57'

In [46]:
df_merged.loc[0,'userType']

'Non-Logged'

In [47]:
user_feature_list = ['userType:Non-Logged']
user_hash = 'fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27288434101e7252ef57'
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

Top 5 recommended items:
        dcfc73c5-c55b-4250-91d5-ba5b450b7b4f
        7667b9be-ab52-4ee8-ab5f-315480da18a5
        79d7a8c1-9b6e-4dc4-8b39-721c7879209d
        06642eb3-87da-48e6-aa43-4781c8b4a10a
        d5afb553-94e7-40b9-84e2-48f28ff93817


In [48]:
user_id_map["fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27288434101e7252ef57"]

0

In [49]:
df_merged.loc[2,'userId']

'17f1083e6079b0f28f7820a6803583d1c1b405c0718b11a18d30b1620f643b23'

In [50]:
df_merged.loc[2,'userType']

'Non-Logged'

In [51]:
sample_recommendation(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

Top 5 recommended items:
        dcfc73c5-c55b-4250-91d5-ba5b450b7b4f
        7667b9be-ab52-4ee8-ab5f-315480da18a5
        79d7a8c1-9b6e-4dc4-8b39-721c7879209d
        06642eb3-87da-48e6-aa43-4781c8b4a10a
        d5afb553-94e7-40b9-84e2-48f28ff93817
